# Rename tensorflow variables
Script to rename variables in tensorflow checkpoint files. Modified to manipulate individual files rather than latest checkpoint in directory.

Credit: https://gist.github.com/batzner/

In [ ]:
import tensorflow as tf
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

In [ ]:
checkpoint_dir = "../experiments/open_field/trial_51/train_data/net_data/main_net/params/"
#checkpoint_dir = "/home/james/Desktop/tmp/"
#checkpoint = tf.train.get_checkpoint_state(checkpoint_dir)
checkpoint = checkpoint_dir + "train_model-200"
print_tensors_in_checkpoint_file(file_name=checkpoint, tensor_name='', all_tensors=False, all_tensor_names=True)

In [ ]:
replace_from = "main_network/rnn/lstm_cell/biases"
replace_to = "main_network/rnn/lstm_cell/bias"
add_prefix = False
dry_run = False

with tf.Session() as sess:
    for var_name, _ in tf.contrib.framework.list_variables(checkpoint):
        # Load the variable
        var = tf.contrib.framework.load_variable(checkpoint, var_name)

        # Set the new name
        new_name = var_name
        if None not in [replace_from, replace_to]:
            new_name = new_name.replace(replace_from, replace_to)
        if add_prefix:
            new_name = add_prefix + new_name

        if dry_run:
            if var_name != new_name:
                print("%s would be renamed to %s." % (var_name, new_name))
            else:
                print("%s would be unchanged." % (var_name))
        else:
            if var_name != new_name:
                print("Renaming %s to %s." % (var_name, new_name))
            else:
                print("Leaving %s unchanged." % (var_name))
            # Rename the variable (MUST do regardless of whether new name is different)
            var = tf.Variable(var, name=new_name)

    if not dry_run:
        # Save the variables
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        saver.save(sess, checkpoint)

print("\nCheckpoint updated!")